In [1]:
import sklearn.datasets as skds
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.models import Model

C:\Users\salumuri\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
file=r'C:\Users\salumuri\Downloads\sentiment labelled sentences\imdb_labelled.txt'

In [3]:
f=open(file,'r')
text=[]
sentiment=[]
for line in f:
    temp=line.strip()
    text.append(temp.split('\t')[0])
    sentiment.append(int(temp.split('\t')[1]))
f.close()
print(text[0],sentiment[0])

A very, very, very slow-moving, aimless movie about a distressed, drifting young man.   0


In [4]:
imdb=pd.DataFrame({'review':text,'sentiment':sentiment})

In [5]:
imdb_train,imdb_test=train_test_split(imdb,test_size=0.2,random_state=2)

In [6]:
x_train=imdb_train['review']
y_train=imdb_train['sentiment']
x_test=imdb_test['review']
y_test=imdb_test['sentiment']

In [7]:
sent_lens=[]
for sent in imdb_train['review']:
    sent_lens.append(len(word_tokenize(sent)))

In [8]:
max(sent_lens)

80

In [9]:
np.quantile(sent_lens,0.95)

37.049999999999955

In [10]:
# We can see that 95% review text are of lengths less than or equal to 37. We'll keep the max length to 30

In [11]:
max_len = 30
tok = Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(x_train)
sequences_train = tok.texts_to_sequences(x_train)

In [12]:
sequences_train

[[128, 5, 589],
 [1, 37, 16, 1003, 332, 40, 6, 117, 5, 246, 1004],
 [141, 2, 62, 8],
 [8, 423, 3, 1005, 279, 333, 280, 4, 1, 424, 2, 1006, 4, 1, 56, 590],
 [6, 5, 183, 20, 4, 1, 24, 1007],
 [591,
  425,
  426,
  3,
  43,
  105,
  129,
  4,
  3,
  1008,
  1009,
  1010,
  1011,
  1012,
  210,
  592],
 [20,
  57,
  247,
  7,
  82,
  1013,
  1014,
  1015,
  28,
  53,
  165,
  26,
  15,
  1016,
  593,
  2,
  1017,
  4,
  1018,
  18,
  1,
  1019,
  594,
  88,
  12,
  23,
  110,
  83],
 [27, 24, 53, 21, 23, 24],
 [38, 595, 38, 1020, 27, 596, 10, 1, 597, 5, 130, 598, 2, 599],
 [36, 427, 600, 106, 35, 10, 601, 59, 2, 1, 1021, 1022, 19, 142],
 [1, 30, 248, 68, 211, 95, 2, 1023, 1024, 89, 28, 143],
 [20, 4, 1, 212, 428, 4, 26, 49],
 [18,
  60,
  429,
  1025,
  15,
  1026,
  2,
  602,
  18,
  426,
  3,
  1027,
  2,
  603,
  334,
  54,
  3,
  1028,
  4,
  1029,
  106,
  335,
  166,
  213,
  25,
  121,
  1,
  212,
  249,
  4,
  3,
  11,
  14,
  90,
  91,
  604,
  28,
  1030,
  1031,
  16,
  335,
  4

In [13]:
tok.index_word

{1: 'the',
 2: 'and',
 3: 'a',
 4: 'of',
 5: 'is',
 6: 'this',
 7: 'i',
 8: 'it',
 9: 'to',
 10: 'in',
 11: 'movie',
 12: 'was',
 13: 'film',
 14: 'that',
 15: 'for',
 16: 'as',
 17: 'with',
 18: 'but',
 19: 'are',
 20: 'one',
 21: "it's",
 22: 'on',
 23: 'just',
 24: 'bad',
 25: 'you',
 26: 'all',
 27: 'so',
 28: 'very',
 29: 'not',
 30: 'good',
 31: 'at',
 32: 'be',
 33: 'about',
 34: 'an',
 35: 'like',
 36: 'there',
 37: 'acting',
 38: 'his',
 39: 'has',
 40: 'from',
 41: 'great',
 42: 'by',
 43: 'really',
 44: 'out',
 45: 'if',
 46: 'even',
 47: 'were',
 48: 'he',
 49: 'time',
 50: 'or',
 51: 'my',
 52: 'who',
 53: 'well',
 54: 'story',
 55: 'have',
 56: 'characters',
 57: 'more',
 58: 'most',
 59: 'movies',
 60: 'when',
 61: 'made',
 62: 'see',
 63: 'plot',
 64: "don't",
 65: 'ever',
 66: 'only',
 67: 'no',
 68: 'also',
 69: 'best',
 70: 'they',
 71: 'what',
 72: 'because',
 73: 'how',
 74: 'do',
 75: 'think',
 76: 'than',
 77: 'real',
 78: 'which',
 79: 'seen',
 80: 'me',
 81: 'y

In [14]:
vocab_len=len(tok.index_word.keys())

In [15]:
vocab_len

2715

In [16]:
sequences_matrix_train = sequence.pad_sequences(sequences_train,maxlen=max_len)

In [17]:
sequences_matrix_train

array([[   0,    0,    0, ...,  128,    5,  589],
       [   0,    0,    0, ...,    5,  246, 1004],
       [   0,    0,    0, ...,    2,   62,    8],
       ...,
       [   0,    0,    0, ...,    0,   62,    8],
       [   0,    0,    2, ...,    2, 2713, 2714],
       [   0,    0,    0, ...,   35,    6,   20]])

In [18]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(vocab_len+1,500,input_length=max_len,mask_zero=True)(inputs) #vocab_length+1 is default and 500 is dimensions.
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [19]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer='adam',
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 30)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 30, 500)           1358000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                144640    
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)  

In [20]:
sequences_test = tok.texts_to_sequences(x_test)
sequences_matrix_test = sequence.pad_sequences(sequences_test,
                                               maxlen=max_len)

In [21]:
model.fit(sequences_matrix_train,y_train,batch_size=50,epochs=50,validation_data=(sequences_matrix_test,y_test))

Instructions for updating:
Use tf.cast instead.
Train on 800 samples, validate on 200 samples
Epoch 1/50
800/800 [==============================] - 6s 7ms/step - loss: 0.6921 - acc: 0.5100 - val_loss: 0.6880 - val_acc: 0.5900
Epoch 2/50
800/800 [==============================] - 3s 4ms/step - loss: 0.6430 - acc: 0.7812 - val_loss: 0.5935 - val_acc: 0.7400
Epoch 3/50
800/800 [==============================] - 3s 4ms/step - loss: 0.3368 - acc: 0.9050 - val_loss: 0.4720 - val_acc: 0.7900
Epoch 4/50
800/800 [==============================] - 3s 4ms/step - loss: 0.1009 - acc: 0.9725 - val_loss: 0.5620 - val_acc: 0.7850
Epoch 5/50
800/800 [==============================] - 3s 4ms/step - loss: 0.0298 - acc: 0.9938 - val_loss: 0.8205 - val_acc: 0.7650
Epoch 6/50
800/800 [==============================] - 3s 4ms/step - loss: 0.0119 - acc: 0.9975 - val_loss: 1.1465 - val_acc: 0.7500
Epoch 7/50
800/800 [==============================] - 3s 4ms/step - loss: 0.0040 - acc: 1.0000 - val_loss: 0.9344 

In [22]:
predictions=model.predict(sequences_matrix_test)

In [23]:
from sklearn.metrics import roc_auc_score

In [24]:
roc_auc_score(y_test,predictions)

0.8459812321501429